In [83]:
from random import random, randint
from time import sleep

import chess
import pandas as pd
import torch as T
from tqdm import tqdm, trange

In [73]:
tqdm.pandas()
games = pd.read_csv('mio.txt', header=None)
games.head()

,0,1
0,6qn/p5kp/PpQ2rp1/2n1NbP1/1PP2P1P/B1P1P3/8/7K w...,c6f6
1,rnb1r3/qp3k2/3bp2p/p1pP1p2/1PP2P2/P2B1nQ1/3P1N...,g3g7
2,4k3/R7/4K3/2BP4/6P1/7p/7N/4R3 w - - 6 106,a7a8
3,4k3/3r3p/2ppB2P/bP3Qp1/5RP1/PP6/5B2/7K w - - 3 59,f5f8
4,3k4/5P2/1p2P1p1/r3p2n/P3Pbq1/3b4/8/2R4K w - - ...,f7f8q


In [74]:
# parameters
#  1    Side to move
# 12    Position of each piece (both sides)
# 12    to squares of each pieces (both sides)
bit_layers = 1 + \
            12 + \
            12

def create_input(board):
    posbits = chess.SquareSet(board.turn).tolist()
    
    for piece in [chess.PAWN, chess.KNIGHT, chess.BISHOP, chess.ROOK, chess.QUEEN, chess.KING]:
        posbits += board.pieces(piece, chess.WHITE).tolist()

    for piece in [chess.PAWN, chess.KNIGHT, chess.BISHOP, chess.ROOK, chess.QUEEN, chess.KING]:
        posbits += board.pieces(piece, chess.BLACK).tolist()
        
    
    # all attack squares
    to_queen_sqs = chess.SquareSet()
    to_rook_sqs = chess.SquareSet()
    to_bishop_sqs = chess.SquareSet()
    to_knight_sqs = chess.SquareSet()
    to_king_sqs = chess.SquareSet()
    to_pawn_sqs = chess.SquareSet()
    for move in board.legal_moves:
        if board.san(move)[0]=='Q':
            to_queen_sqs.add(move.to_square)
        elif board.san(move)[0]=='R':
            to_rook_sqs.add(move.to_square)
        elif board.san(move)[0]=='B':
            to_bishop_sqs.add(move.to_square)
        elif board.san(move)[0]=='N':
            to_knight_sqs.add(move.to_square)
        elif board.san(move)[0]=='K' or board.san(move)[0]=='O':
            to_king_sqs.add(move.to_square)
        else:
            to_pawn_sqs.add(move.to_square)
    posbits += to_queen_sqs.tolist()+to_rook_sqs.tolist()+to_bishop_sqs.tolist()+to_knight_sqs.tolist()+to_king_sqs.tolist()+to_pawn_sqs.tolist()
    
    # all opponent attack squares
    board.turn = not board.turn
    to_queen_sqs = chess.SquareSet()
    to_rook_sqs = chess.SquareSet()
    to_bishop_sqs = chess.SquareSet()
    to_knight_sqs = chess.SquareSet()
    to_king_sqs = chess.SquareSet()
    to_pawn_sqs = chess.SquareSet()
    for move in board.legal_moves:
        if board.san(move)[0]=='Q':
            to_queen_sqs.add(move.to_square)
        elif board.san(move)[0]=='R':
            to_rook_sqs.add(move.to_square)
        elif board.san(move)[0]=='B':
            to_bishop_sqs.add(move.to_square)
        elif board.san(move)[0]=='N':
            to_knight_sqs.add(move.to_square)
        elif board.san(move)[0]=='K' or board.san(move)[0]=='O':
            to_king_sqs.add(move.to_square)
        else:
            to_pawn_sqs.add(move.to_square)
    posbits += to_queen_sqs.tolist()+to_rook_sqs.tolist()+to_bishop_sqs.tolist()+to_knight_sqs.tolist()+to_king_sqs.tolist()+to_pawn_sqs.tolist()
    board.turn = not board.turn
    
    #en passant square
    #posbits += (chess.SquareSet(chess.BB_SQUARES[board.ep_square]) if board.ep_square else chess.SquareSet()).tolist()
    
    x = T.tensor(posbits, dtype = T.float32)
    x = x.reshape([bit_layers,8,8])
    return x


In [75]:
games['board'] = games[0].progress_apply(chess.Board)
games['inbits'] = games['board'].progress_apply(create_input)
games.head()

100%|██████████| 30012/30012 [03:17<00:00, 151.61it/s]


,0,1,board,inbits
0,6qn/p5kp/PpQ2rp1/2n1NbP1/1PP2P1P/B1P1P3/8/7K w...,c6f6,. . . . . . q n\np . . . . . k p\nP p Q . . r ...,"[[[tensor(1.), tensor(0.), tensor(0.), tensor(..."
1,rnb1r3/qp3k2/3bp2p/p1pP1p2/1PP2P2/P2B1nQ1/3P1N...,g3g7,r n b . r . . .\nq p . . . k . .\n. . . b p . ...,"[[[tensor(1.), tensor(0.), tensor(0.), tensor(..."
2,4k3/R7/4K3/2BP4/6P1/7p/7N/4R3 w - - 6 106,a7a8,. . . . k . . .\nR . . . . . . .\n. . . . K . ...,"[[[tensor(1.), tensor(0.), tensor(0.), tensor(..."
3,4k3/3r3p/2ppB2P/bP3Qp1/5RP1/PP6/5B2/7K w - - 3 59,f5f8,. . . . k . . .\n. . . r . . . p\n. . p p B . ...,"[[[tensor(1.), tensor(0.), tensor(0.), tensor(..."
4,3k4/5P2/1p2P1p1/r3p2n/P3Pbq1/3b4/8/2R4K w - - ...,f7f8q,. . . k . . . .\n. . . . . P . .\n. p . . P . ...,"[[[tensor(1.), tensor(0.), tensor(0.), tensor(..."


In [33]:

games['move'] = games[1].apply(chess.Move.from_uci)
games = games.drop([0,1], axis=1)
games.head()

,board,move,legal_moves
2,. . . . k . . .\nR . . . . . . .\n. . . . K . ...,a7a8,"[a7a8, a7h7, a7g7, a7f7, a7e7, a7d7, a7c7, a7b..."
3,. . . . k . . .\n. . . r . . . p\n. . p p B . ...,f5f8,"[e6g8, e6f7, e6d7, e6d5, e6c4, f5f8, f5h7, f5f..."
4,. . . k . . . .\n. . . . . P . .\n. p . . P . ...,f7f8q,"[c1c8, c1c7, c1c6, c1c5, c1c4, c1c3, c1c2, c1g..."
5,. . R . r k r .\n. Q . . . . . .\n. . . . . . ...,f4e6,"[c8e8, c8d8, c8b8, c8a8, c8c7, c8c6, c8c5, c8c..."
6,r . b k r . . .\n. p . p p . n p\np . . . . . ...,b5b6,"[b5d7, b5b7, b5c6, b5b6, b5a6, b5d5, b5c5, b5a..."


In [85]:
with trange(10) as t:
    for i in t:
        # Description will be displayed on the left
        t.set_description('GEN %i' % i)
        # Postfix will be displayed on the right,
        # formatted automatically based on argument's datatype
        t.set_postfix(loss=random(), gen=randint(1,999), str='h',
                      lst=[1, 2])
        sleep(1)

GEN 9: 100%|██████████| 10/10 [00:10<00:00,  1.01s/it, gen=642, loss=0.314, lst=[1, 2], str=h]


In [22]:
#games['board'][12]

In [29]:
games['legal_moves'] = games['board'].apply(lambda x : list(x.legal_moves))
games.head()

,0,1,board,move,legal_moves
0,6qn/p5kp/PpQ2rp1/2n1NbP1/1PP2P1P/B1P1P3/8/7K w...,c6f6,. . . . . . q n\np . . . . . k p\nP p Q . . r ...,c6f6,"[c6e8, c6c8, c6a8, c6d7, c6c7, c6b7, c6f6, c6e..."
1,rnb1r3/qp3k2/3bp2p/p1pP1p2/1PP2P2/P2B1nQ1/3P1N...,g3g7,r n b . r . . .\nq p . . . k . .\n. . . b p . ...,g3g7,"[g3g8, g3g7, g3g6, g3g5, g3h4, g3g4, g3h3, g3f..."
2,4k3/R7/4K3/2BP4/6P1/7p/7N/4R3 w - - 6 106,a7a8,. . . . k . . .\nR . . . . . . .\n. . . . K . ...,a7a8,"[a7a8, a7h7, a7g7, a7f7, a7e7, a7d7, a7c7, a7b..."
3,4k3/3r3p/2ppB2P/bP3Qp1/5RP1/PP6/5B2/7K w - - 3 59,f5f8,. . . . k . . .\n. . . r . . . p\n. . p p B . ...,f5f8,"[e6g8, e6f7, e6d7, e6d5, e6c4, f5f8, f5h7, f5f..."
4,3k4/5P2/1p2P1p1/r3p2n/P3Pbq1/3b4/8/2R4K w - - ...,f7f8q,. . . k . . . .\n. . . . . P . .\n. p . . P . ...,f7f8q,"[c1c8, c1c7, c1c6, c1c5, c1c4, c1c3, c1c2, c1g..."


In [36]:
games['legal_moves_count'] = games['legal_moves'].apply(lambda x : len(x))
games.head()

,board,move,legal_moves,legal_moves_count
2,. . . . k . . .\nR . . . . . . .\n. . . . K . ...,a7a8,"[a7a8, a7h7, a7g7, a7f7, a7e7, a7d7, a7c7, a7b...",43
3,. . . . k . . .\n. . . r . . . p\n. . p p B . ...,f5f8,"[e6g8, e6f7, e6d7, e6d5, e6c4, f5f8, f5h7, f5f...",40
4,. . . k . . . .\n. . . . . P . .\n. p . . P . ...,f7f8q,"[c1c8, c1c7, c1c6, c1c5, c1c4, c1c3, c1c2, c1g...",18
5,. . R . r k r .\n. Q . . . . . .\n. . . . . . ...,f4e6,"[c8e8, c8d8, c8b8, c8a8, c8c7, c8c6, c8c5, c8c...",51
6,r . b k r . . .\n. p . p p . n p\np . . . . . ...,b5b6,"[b5d7, b5b7, b5c6, b5b6, b5a6, b5d5, b5c5, b5a...",38


In [38]:
games['turn'] = games['board'].apply(lambda x : x.turn)
games.head()

,board,move,legal_moves,legal_moves_count,turn
2,. . . . k . . .\nR . . . . . . .\n. . . . K . ...,a7a8,"[a7a8, a7h7, a7g7, a7f7, a7e7, a7d7, a7c7, a7b...",43,True
3,. . . . k . . .\n. . . r . . . p\n. . p p B . ...,f5f8,"[e6g8, e6f7, e6d7, e6d5, e6c4, f5f8, f5h7, f5f...",40,True
4,. . . k . . . .\n. . . . . P . .\n. p . . P . ...,f7f8q,"[c1c8, c1c7, c1c6, c1c5, c1c4, c1c3, c1c2, c1g...",18,True
5,. . R . r k r .\n. Q . . . . . .\n. . . . . . ...,f4e6,"[c8e8, c8d8, c8b8, c8a8, c8c7, c8c6, c8c5, c8c...",51,True
6,r . b k r . . .\n. p . p p . n p\np . . . . . ...,b5b6,"[b5d7, b5b7, b5c6, b5b6, b5a6, b5d5, b5c5, b5a...",38,True


In [47]:
'Average number moves that white has: ' +str(int(round(games['legal_moves_count'].mean())))

'Average number moves that white has: 38'

In [50]:
games['Queen'] = games['board'].apply(lambda x: x.pieces(chess.QUEEN, chess.WHITE))
games.head(10)

,board,move,legal_moves,legal_moves_count,turn,Queen
2,. . . . k . . .\nR . . . . . . .\n. . . . K . ...,a7a8,"[a7a8, a7h7, a7g7, a7f7, a7e7, a7d7, a7c7, a7b...",43,True,()
3,. . . . k . . .\n. . . r . . . p\n. . p p B . ...,f5f8,"[e6g8, e6f7, e6d7, e6d5, e6c4, f5f8, f5h7, f5f...",40,True,(37)
4,. . . k . . . .\n. . . . . P . .\n. p . . P . ...,f7f8q,"[c1c8, c1c7, c1c6, c1c5, c1c4, c1c3, c1c2, c1g...",18,True,()
5,. . R . r k r .\n. Q . . . . . .\n. . . . . . ...,f4e6,"[c8e8, c8d8, c8b8, c8a8, c8c7, c8c6, c8c5, c8c...",51,True,"(19, 49)"
6,r . b k r . . .\n. p . p p . n p\np . . . . . ...,b5b6,"[b5d7, b5b7, b5c6, b5b6, b5a6, b5d5, b5c5, b5a...",38,True,(33)
7,. . . . . . . N\n. . . . . . . .\n. . . . . . ...,h8g6,"[h8f7, h8g6, f4g6, f4e6, f4h5, f4d5, f4h3, f4d...",24,True,()
8,. . . . . . R .\nB . . . . . . .\n. . . . . . ...,f1h1,"[g8h8, g8f8, g8e8, g8d8, g8c8, g8b8, g8a8, g8g...",43,True,()
9,. . . . . k . .\n. . Q . . q . .\n. . b p . . ...,c7f7,"[c7d8, c7c8, c7b8, c7f7, c7e7, c7d7, c7b7, c7a...",49,True,(50)
10,. . . . . . b k\n. . R . . . . .\n. . . . . . ...,h4g6,"[c7c8, c7h7, c7g7, c7f7, c7e7, c7d7, c7b7, c7a...",30,True,()
11,. k . . . . . .\n. . . . . R . .\n. . . R . . ...,d6d8,"[f7f8, f7h7, f7g7, f7e7, f7d7, f7c7, f7b7, f7a...",32,True,()


In [51]:
games['best_move_from'] = games['move'].apply(lambda x: x.from_square)
games.head()

,board,move,legal_moves,legal_moves_count,turn,Queen,best_move_from
2,. . . . k . . .\nR . . . . . . .\n. . . . K . ...,a7a8,"[a7a8, a7h7, a7g7, a7f7, a7e7, a7d7, a7c7, a7b...",43,True,(),48
3,. . . . k . . .\n. . . r . . . p\n. . p p B . ...,f5f8,"[e6g8, e6f7, e6d7, e6d5, e6c4, f5f8, f5h7, f5f...",40,True,(37),37
4,. . . k . . . .\n. . . . . P . .\n. p . . P . ...,f7f8q,"[c1c8, c1c7, c1c6, c1c5, c1c4, c1c3, c1c2, c1g...",18,True,(),53
5,. . R . r k r .\n. Q . . . . . .\n. . . . . . ...,f4e6,"[c8e8, c8d8, c8b8, c8a8, c8c7, c8c6, c8c5, c8c...",51,True,"(19, 49)",29
6,r . b k r . . .\n. p . p p . n p\np . . . . . ...,b5b6,"[b5d7, b5b7, b5c6, b5b6, b5a6, b5d5, b5c5, b5a...",38,True,(33),33


In [58]:
games['Black_King'] = games['board'].apply(lambda x: x.pieces(chess.KING, chess.BLACK))
games['Black_King'] = games['Black_King'].apply(lambda x: x[0])
games.head(10)

TypeError: 'SquareSet' object does not support indexing